# Shrinking of the Space

Important note:
This program has to be a SageMath Notebook to run the code 

Preparation for the oracle implementation: Solve the XOR SAT problem 
1. Get Kernel of matrix A mod 2 like constructed in the paper
2. Get solution of (Ax = b) mod 2 with inhomogenous part b like constructed in the paper

In [10]:
import numpy as np 

In [11]:
p = 3
q = 1
eps = 1e-10

# get q-in-p-SAT problem instance (Variables start with 1 for x_1)
constraints = np.array([[1, -2, 3], [2, -3, 4], [-3, -4, 5]])

M = len(constraints) # number constraints
n = np.max(constraints.max()) # number variables

# change Instance to XOR instance
A = np.zeros((M, n), dtype = 'int')
b = np.zeros(M, dtype = 'int')

# A_ij = 1 if variable x_j is in clause i
for row, clause in enumerate(constraints):
    A[row, np.abs(clause) - 1] = 1
    
    v_a = 0
    for var in clause:
        if var < 0:
            v_a += 1
    
    # q + number of negated variables in clause
    b[row] = (q + v_a) % 2

print(A)
print(b)

[[1 1 1 0 0]
 [0 1 1 1 0]
 [0 0 1 1 1]]
[0 0 1]


In [12]:
# get kernel of matrix
A = matrix(GF(2), A)
b = vector(GF(2), b)
null_space = A.right_kernel()
null_space = null_space.basis_matrix()
print("Null space of A:\n", null_space)

Null space of A:
 [1 0 1 1 0]
[0 1 1 0 1]


In [13]:
# get a binary solution of (Ax = b) mod 2
xi_bar = A \ b
print('The value of xi_bar is:', xi_bar)

The value of xi_bar is: (0, 1, 1, 0, 0)


In [14]:
xi_bar = np.array(xi_bar)
null_space = np.array(null_space)

In [15]:
# print out the variables we need properly so we can copy and use them in the quantum part
xi_bar_repr = repr(xi_bar)
null_space_repr = repr(null_space)

# Copy the string representation to another document or location
# For example, you can print it and copy the output from the console
print(null_space_repr)
print(xi_bar_repr)

array([[1, 0, 1, 1, 0],
       [0, 1, 1, 0, 1]], dtype=object)
array([0, 1, 1, 0, 0], dtype=object)


We use the null_space and xi_bar to create our circuit. It shrinks the solution space to all the possible valid assignments of the XOR-SAT problem. In the section below, you can see all possible variable assignments for instances which solve the XOR-SAT problem. 

In [16]:
import itertools

In [17]:
# generate all possible binary combinations to get all linear combinations of vectors x
k = len(null_space)
binary_combinations = list(itertools.product([0, 1], repeat=k))
print(binary_combinations)

[(0, 0), (0, 1), (1, 0), (1, 1)]


In [18]:
# create subspace with all possible solutions to the XOR-SAT problem
subspace = np.tile(xi_bar, (2 **k, 1)) # add xi_bar to every clause

for i, binary_combi in enumerate(binary_combinations):
    for pos, var in enumerate(binary_combi):
        subspace[i, :] += var * null_space[pos]

# we get a list of all possible solutions for p-XOR problem [variable assignment 1, variable assignment 2, ,...] 
print(subspace)
for s in subspace:
    print('Test:', np.dot(A, s))

[[0 1 1 0 0]
 [0 0 0 0 1]
 [1 1 0 1 0]
 [1 0 1 1 1]]
Test: [0 0 1]
Test: [0 0 1]
Test: [0 0 1]
Test: [0 0 1]
